<!-- SPDX-License-Identifier: CC-BY-4.0 -->
<!-- Copyright Contributors to the ODPi Egeria project 2024. -->

![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/main/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Workbook

# Cataloguing and surveying PostgreSQL Servers

## Introduction

A [PostgreSQL server](https://www.postgresql.org/) is an open source technology that supports multiple relational databases.  The contents of these databases are organized into schemas, tables and columns giving a 4 level name space.  The default schema for a database is called **public** and there is also a special database called **pgcatalog** that provides a metadata catalog for the other databases managed by the servers.  PostgreSQL is a mature, fast and flexible database that is widely use in the industry.   Egeria itself uses a PostgreSQL database as an option for its metadata repository, and for the management of its observability data.

Egeria also has support for understanding and governing postgreSQL databases whatever their use.  This notebook takes you through these capabilities.

First lets initialize **pyegeria**.

In [ ]:
# Initialize pyegeria

%run ../../pyegeria/initialize-pyegeria.ipynb


In [ ]:

egeria_tech = EgeriaTech(view_server, url, user_id, user_pwd)
token = egeria_tech.create_egeria_bearer_token()


---

## Loading support for PostgreSQL Servers

The definition of the postgres connectors, templates and associated reference data are loaded via a [Content Pack](https://egeria-project.org/content-packs/) called `PostgresContentPack.omarchive`.  The content pack can be loaded multiple times without ill-effect so run the following command to make sure it is loaded.

---

In [ ]:

from pyegeria import ClassificationManager

classification_manager = ClassificationManager(view_server, url, user_id, user_pwd)
token = classification_manager.create_egeria_bearer_token()

elements = classification_manager.find_elements_by_property_value(property_value="PostgreSQL", property_names=['displayName'], metadata_element_type_name="GovernanceActionProcess")
if type(elements) == str:
    print (elements)
else:
    for element in elements:
        if element:
            properties=element.get('properties')
            if properties:
                qualifiedName=properties.get('qualifiedName')
                description=properties.get('description')
                print('* ' + qualifiedName + ' - ' + description)


----

## Survey a PostgreSQL Server

Egeria's PostgreSQL support includes the ability to survey the contents of a PostgreSQL Server to discover the databases that is manages.  This command creates a description of the PostgreSQL Server and runs a survey to understand its contents.  A summary of the survey results can be found in /distribution-hub/surveys.

---

In [ ]:

from pyegeria import GovernanceOfficer

governance_officer=GovernanceOfficer(view_server, url, user_id, user_pwd)
token = governance_officer.create_egeria_bearer_token()

createAndSurveyProcessName="PostgreSQLServer:CreateAndSurveyGovernanceActionProcess"

process_guid = classification_manager.get_element_guid_by_unique_name(createAndSurveyProcessName)
process_graph = governance_officer.get_governance_process_graph(process_guid)

print_governance_action_process_graph(process_graph)


In [ ]:

from pyegeria import AutomatedCuration
automated_curation=AutomatedCuration(view_server, url, user_id, user_pwd)
token = automated_curation.create_egeria_bearer_token()

# 5442
requestParameters = {
    "serverName" : "LocalPostgreSQL1",
    "hostIdentifier" : "localhost",
    "portNumber" : "5432",
    "secretsStorePathName" : "loading-bay/secrets/integration.omsecrets",
    "secretsCollectionName" : "PostgreSQL Server:LocalPostgreSQL1",
    "versionIdentifier" : "1.0",
    "description" : "PostgreSQL database in egeria-workspaces."
}

automated_curation.initiate_gov_action_process(createAndSurveyServerName, None, None, None, requestParameters, None, None)



----

This next command catalogs the databases it finds on the PostgreSQL Server.

----

In [ ]:

createAndCatalogServerName="PostgreSQLServer::CreateAsCatalogTargetGovernanceActionProcess"

process_guid = classification_manager.get_element_guid_by_unique_name(createAndCatalogServerName)
process_graph = governance_officer.get_governance_process_graph(process_guid)

print_governance_action_process_graph(process_graph)


In [ ]:

# 5442
requestParameters = {
    "serverName" : "LocalPostgreSQL1",
    "hostIdentifier" : "localhost",
    "portNumber" : "5432",
    "secretsStorePathName" : "loading-bay/secrets/integration.omsecrets",
    "secretsCollectionName" : "PostgreSQL Server:LocalPostgreSQL1",
    "versionIdentifier" : "1.0",
    "description" : "PostgreSQL database in egeria-workspaces."
}

egeria_tech.initiate_gov_action_process(createAndCatalogServerName, None, None, None, requestParameters, None, None)

